# NASA-FIRMS Data Scraping via API
---
NRT VIIRS 375 m Active Fire product VNP14IMGT distributed from NASA FIRMS. doi:10.5067/FIRMS/VIIRS/VNP14IMGT_NRT.002

https://firms.modaps.eosdis.nasa.gov/content/academy/data_api/firms_api_use.html

In [49]:
# Dependencies
import pandas as pd
import fns as f
import time
from api_key import nasa_key

## 1 Set Variables and Parameters
---

### 1.1 Set Dates and Range

In [30]:
# Set year
yr = 2022
# Create date range to pull fire data
dt = [f'{str(x.date())}' for x in pd.date_range(str(yr)+"-01-01", str(yr)+"-12-31", freq="D")]
# Create [date,range] list for (Date format is %Y-%m-%d, Range is 1-10 days)
dt_rng = []
for i in range(0,int(f.roundby(len(dt)/10+0.5,1))):
    dt_rng.append([dt[10*i],min(10,len(dt)-10*i)])
    i+=1
# Check last entry
dt_rng[-1]

['2022-12-27', 5]

### 1.2 Set Fixed API Parameters

In [25]:
base_url='https://firms.modaps.eosdis.nasa.gov/api/area/csv/'

In [ ]:
model = 'VIIRS_SNPP_SP' # VIIRS_SNPP_SP,2012-01-20,2024-01-31 (VIIRS Suomi-NPP Standard Processing)
# model = 'VIIRS_SNPP_NRT' # VIIRS_SNPP_NRT,2024-02-01,2024-07-09 (VIIRS Suomi-NPP Near Real Time)
# model = 'VIIRS_NOAA20_NRT' # VIIRS_NOAA20_NRT, 2019-12-04,2023-08-02

In [ ]:
# California BoundingBox(W,S,E,N): -124.482003000119, 32.5288320002052, -114.131210999644, 42.0095030001829
bbox = f'{f.CAbounds(1)[0]},{f.CAbounds(1)[1]},{f.CAbounds(1)[2]},{f.CAbounds(1)[3]}'

## 2 API Requests
---

### 2.1 Initial Loop For Requests

In [ ]:
# Create list to catch failed dates
dt_failed = []
# Loop through requests and output results to csv
for dr in dt_rng:
    csv_url=f'{base_url}{nasa_key}/{model}/{bbox}/{dr[1]}/{dr[0]}'
    try: 
        pd.read_csv(csv_url).to_csv(f'../outputs/nasa-firms/{yr}/{dr[0]}_{dr[1]}.csv',index=False)
    except:
        print(dr)
        dt_failed.append(dr)
    time.sleep(1)

### 2.2 Loop for Failed Requests

In [11]:
# Check for failed requests
dt_failed

In [ ]:
# Loop through failed requests and output results to csv
for dr in dt_failed:
    print(dr[0])
    csv_url=f'{base_url}{nasa_key}/{model}/{bbox}/{dr[1]}/{dr[0]}'
    try: 
        pd.read_csv(csv_url).to_csv(f'../outputs/nasa-firms/{yr}/{dr[0]}_{dr[1]}.csv',index=False)
    except:
        print(dr)
    time.sleep(1)

### 2.3 Request by Date

In [36]:
dr=['2022-07-10',10]
csv_url=f'{base_url}{nasa_key}/{model}/{bbox}/{dr[1]}/{dr[0]}'
df = pd.read_csv(csv_url)

In [46]:
df.tail(1)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
1336,40.96521,-123.589,329.82,0.45,0.39,2022-08-18,2123,N20,VIIRS,l,2.0NRT,300.93,5.99,D


In [38]:
df.to_csv(f'../outputs/nasa-firms/{yr}/{dr[0]}_{dr[1]}.csv',index=False)